In [1]:
from fit.datamodules.super_res.SRecDataModule import CelebASResFourierTargetDataModule
from fit.utils import convert2FC
from fit.utils.tomo_utils import get_img_coords_pol

from fit.modules.SResTransformerModule import SResTransformerModule

from matplotlib import pyplot as plt

import torch

import numpy as np

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint

In [2]:
seed_everything(22122020)

22122020

In [3]:
dm = CelebASResFourierTargetDataModule(root_dir='/data/celeb/', batch_size=32)
dm.setup()

In [4]:
r, phi, flatten_order, order = get_img_coords_pol(img_shape=dm.gt_shape, det_len=dm.gt_shape)

In [5]:
n_heads = 8
d_query = 32

In [6]:
model = SResTransformerModule(d_model=n_heads*d_query, 
                              img_shape=dm.gt_shape,
                              y_coords_img=r,
                              x_coords_img=phi,
                              dst_flatten_order=flatten_order,
                              dst_order=order,
                              loss='prod',
                              lr=0.0001, weight_decay=0.01, n_layers=8,
                              n_heads=n_heads, d_query=d_query, dropout=0.1, attention_dropout=0.1)

In [7]:
trainer = Trainer(max_epochs=100, 
                  gpus=1,
                  checkpoint_callback=ModelCheckpoint(
                                            filepath=None,
                                            save_top_k=1,
                                            verbose=False,
                                            save_last=True,
                                            monitor='Train/avg_val_loss',
                                            mode='min',
                                            prefix='best_val_loss_'
                                        ), 
                  deterministic=True)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [8]:
trainer.fit(model, datamodule=dm);


  | Name | Type                 | Params
----------------------------------------------
0 | sres | SResTransformerTrain | 6.3 M 
----------------------------------------------
6.3 M     Trainable params
0         Non-trainable params
6.3 M     Total params
/home/tbuchhol/Programs/miniconda3/envs/ft/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 56 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/tbuchhol/.local/lib/python3.7/site-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/tbuchhol/Programs/miniconda3/envs/ft/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 56 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/tbuchhol/Gitrepos/FourierImageTransformer/fit/utils/RAdam.py:119: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


Epoch    65: reducing learning rate of group 0 to 5.0000e-05.


Epoch    78: reducing learning rate of group 0 to 2.5000e-05.


Epoch    89: reducing learning rate of group 0 to 1.2500e-05.


Saving latest checkpoint...


Epoch   100: reducing learning rate of group 0 to 6.2500e-06.

